# Исследование надежности заемщиков.
**Описание проекта**

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли `семейное положение` и `количество детей` клиента на `факт погашения кредита в срок`. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Обзор данных
Составение представления о входных данных от банка — статистика о платёжеспособности клиентов (путь к файлу: /datasets/data.csv):

In [1]:
# импорт библиотеки pandas
import pandas as pd

# чтение файла с данными и сохранение в df
df = pd.read_csv('/datasets/data.csv')

# получение первых 10 строк таблицы df
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
# вывод информации о df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице `12 столбцов`:
* `children` — количество детей в семье (целые цисла);
* `days_employed` — количество дней трудового стажа (числа с плавающей точкой);  
* `dob_years` — возраст клиента в годах (целые цисла);
* `education` — уровень образования клиента (object);
* `education_id` — идентификатор уровня образования (целые цисла);
* `family_status` — семейное положение (object);
* `family_status_id` — идентификатор семейного положения (целые цисла);
* `gender` — пол клиента (object);  
* `income_type` — тип занятости (object);
* `debt` — имел ли задолженность по возврату кредитов (целые цисла: 0, 1);
* `total_income` — ежемесячный доход (числа с плавающей точкой);
* `purpose` — цель получения кредита (object);

В соответствии с информацией о полученных данных в столбцах **'days_employed'** и **'total_income'** есть пропущенные значения (заполнено **19351** строк при общем количестве строк **21525**).
Нарушения стиля в названиях столбцов нет.

### Шаг 2.1 Заполнение пропусков
* Расчет количества пропусков и поиск соответветсвующих строк.
* Проверка доли пропущенных значений в каждом из столбцов с пропусками;
* Описание пропущенных значений;
* Анализ причин появления пропусков в данных;
* Обоснование необходимости заполнения пропусков медианным значением.

In [3]:
# подсчёт пропусков
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
# отображение первых 10 строк с пропущенными данными:
df[df.isna().any(axis=1)].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


In [5]:
# Доля пропусков в столбце 'days_employed'
df['days_employed'].isna().sum() / len(df['days_employed'].index)

0.10099883855981417

In [6]:
# Доля пропусков в столбце 'total_income'
df['total_income'].isna().sum() / len(df['total_income'].index)

0.10099883855981417

Пропущенные значения относятся к столбцам с данными о трудовом стаже и ежемесячном доходе.

Вероятно отсутствие данных обусловлено ограничениями в конфиденциальности информации или невозможностью их отображения.

Доля пропусков в каждом столбце составлет 10%, что является существенным значением для последующего анализа.

Некоторые значения в столбцах `'days_employed'` и `'total_income'` сильно выделяются среди большинства и среднее значение будет некорректно характеризовать данные, поэтому для заполнения пропусков используются медианные значения.

In [7]:
df.pivot_table(index='income_type', values='days_employed', aggfunc=['count', 'median', 'mean'])

,count,median,mean
,days_employed,days_employed,days_employed
income_type,,,
безработный,2,366413.652744,366413.652744
в декрете,1,-3296.759962,-3296.759962
госслужащий,1312,-2689.368353,-3399.896902
компаньон,4577,-1547.382223,-2111.524398
пенсионер,3443,365213.306266,365003.491245
предприниматель,1,-520.848083,-520.848083
сотрудник,10014,-1574.202821,-2326.499216
студент,1,-578.751554,-578.751554


In [8]:
# Корректировка трудового стажа для безработных и пенсионеров 
df.loc[df['income_type'] == 'безработный', 'days_employed'] = df.loc[df['income_type'] == 'безработный', 'days_employed'] / 24
df.loc[df['income_type'] == 'пенсионер', 'days_employed'] = df.loc[df['income_type'] == 'пенсионер', 'days_employed'] / 24

In [9]:
# Заполнение пропусков медианными значениями
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())
df['total_income'] = df['total_income'].fillna(df['total_income'].median())

# Проверка на отсутствие пропусков
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Шаг 2.2 Проверка данных на аномалии и исправления.
**Проверка значений по столбцам:**

* **Количество детей в семье:**

In [10]:
# Уникальные значения по количеству детей:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Аномальные значения в столбце `'children'`: **-1** и **20**. Проверка количества и правдоподобности данных:

In [11]:
# Проверка количества записей со значением "-1" в столбце 'children'
df[df['children']==-1]['children'].count()

47

Вероятно отрицательные значения - ошибочная запись, корректировка значений без отрицательного знака:

In [12]:
# Замена "-1" на "1" в столбце 'children'
df.loc[df['children']==-1, 'children'] = 1

In [13]:
# Проверка количества записей со значением "20" в столбце 'children'
df[df['children']==20]['children'].count()

76

In [14]:
# Проверка возраста клиентов с количеством детей = 20 шт
df[df['children']==20]['dob_years'].head(10)

606     21
720     44
1074    56
2510    59
2941     0
3302    35
3396    56
3671    23
3697    40
3735    26
Name: dob_years, dtype: int64

У людей с возрастом 20-30 лет маловероятно наличие детей в количестве 20 шт, следовательно данные ошибочные и вместо **20** наиболее корректное значение **2** (ошибочно добавлен 0). Корректировка значений "20" на "2":

In [15]:
# Замена "20" на "2" в столбце 'children'
df.loc[df['children']==20, 'children'] = 2

# Проверка уникальных значений после корректировок:
df['children'].unique()

array([1, 0, 3, 2, 4, 5])

* **Количество дней трудового стажа:**

Присутствуют отрицательные значения. Корректировка:

In [16]:
# Корректировка отрицательных значений в столбце 'days_employed':
df['days_employed'] = df['days_employed'].abs()

# Проверка первых 5 строк таблицы df
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


* **Возраст клиента в годах:**

In [17]:
# Уникальные значения по возрасту клиентов:
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

Аномальные значения в столбце 'dob_years': 0. Вычисление доли и определение необходимости корреткировки значений:

In [18]:
# Доля записей со значением "0" в столбце 'dob_years'
df[df['dob_years']==0]['dob_years'].count() / len(df['dob_years'].index)

0.004692218350754936

Доля записей со значением "0" в столбце с возратом клиетов менее 1%. Данный показатель не участвует в исследовании, но для чистоты эксперимента, произведена корректировка на медианное значение:

In [19]:
# Замена "0" на медианное значение в столбце 'dob_years'
df.loc[df['dob_years']==0, 'dob_years'] = int(df['dob_years'].median())

# Проверка уникальных значений после корректировок:
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75])

* **Уровень образования клиента:**

In [20]:
# Уникальные значения по образованию клиентов:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Присутствуют дубликаты. Корректировка выполнена в п.`0.5 "Шаг 2.4. Удаление дубликатов"`

* **Идентификатор уровня образования:**

In [21]:
# Уникальные значения по идентификатору уровня образованию клиентов:
df['education_id'].unique()

array([0, 1, 2, 3, 4])

Соответствие идентификатора и образования:
* `0` - высшее;
* `1` - среднее;
* `2` - неоконченное высшее;
* `3` - начальное;
* `4` - ученная степень.

* **Семейное положение:**

In [22]:
# Уникальные значения по семейному положению клиентов:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Присутвтует 'Не женат / не замужем' с буквой в верхнем регистре. Корректировка:

In [23]:
# Приведение значений столбца 'family_status' к одному регистру
df['family_status'] = df['family_status'].str.lower()

# Проверка уникальных значений после корреткировки
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

* **Идентификатор семейного положения:**

In [24]:
# Уникальные значения по индетификатору семейного положения клиентов:
df['family_status_id'].unique()

array([0, 1, 2, 3, 4])

Соответствие идентификатора и семейного положения:
* `0` - женат / замужем;
* `1` - гражданский брак;
* `2` - вдовец / вдова;
* `3` - в разводе;
* `4` - не женат / не замужем.

* **Пол клиента:**

In [25]:
# Уникальные значения пола клиентов:
df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

Присутствует аномальное значение `'XNA'`.

In [26]:
# отоброжение строки со значением 'XNA' в столбце 'gender'
df[df['gender']=='XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Пол клиента не является ключевым показателем в исследовании. Возможно пол клиента не относится к 'F' или 'M'. Значение остается без корректировки.

* **Тип занятости:**

In [27]:
# Уникальные значения по типам занятости клиентов:
df['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

* **Задолженность по возврату кредитов:**

In [28]:
# Уникальные значения по задолжности:
df['debt'].unique()

array([0, 1])

* **Ежемесячный доход:**

In [29]:
# Проверка отрицательных значений:
df[df['total_income']<0]['total_income'].count()

0

* **Цель получения кредита:**

In [30]:
# Уникальные значения по целям получения кредита:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Присутвует множество вариаций одного и того же действия. Данные будут использованы при категоризации в пункте `"0.8 Шаг 2.7. Категоризация целей кредита."`

### Шаг 2.3. Изменение типов данных.

In [31]:
# Замена вещественного типа данных в столбце `'total_income'` на целочисленный
df['total_income'] = df['total_income'].astype('int')

# Проверка после корректировки
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

**Неявные дубликаты:**

In [32]:
# Приведение значений столбца 'education' к одному регистру
df['education'] = df['education'].str.lower()

# Проверка уникальных значений после корреткировки
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

**Пояснения:** явные дубликаты найдены и посчитаны с помощью `.duplicated().sum()`. Неявные будликаты были выявлены в ходе проверки данных на аномалии с использованием уникальных значений `unique()`, т.к. основные отличия состояли в разнице регистров, то для устранения неявных дубликатов использовалось приведение к одному регистру `.str.lower()`.

**Явные дубликаты:**

In [33]:
# подсчёт явных дубликатов
df.duplicated().sum()

72

In [34]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
df = df.drop_duplicates().reset_index(drop=True)

# проверка на отсутствие дубликатов
df.duplicated().sum()

0

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

**Формирование дополнительного датафрейма словаря для образования:**

In [35]:
# Создание отдельной таблицы education_dict
education_dict = df[['education', 'education_id']]

# Удаление дубликатов
education_dict = education_dict.drop_duplicates().reset_index(drop=True)

# Проверка
education_dict

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


**Формирование дополнительного датафрейма словаря для семейного положения:**

In [36]:
# Создание отдельной таблицы family_status_dict
family_status_dict = df[['family_status', 'family_status_id']]

# Удаление дубликатов
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)

# Проверка
family_status_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


**Декомпозиция исходного датафрейма:**

In [37]:
# Удаление столбцов 'education' и 'family_status'
df = df.drop('education', 1)
df = df.drop('family_status', 1)

# Проверка после удаления столбцов
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21453 non-null  int64  
 1   days_employed     21453 non-null  float64
 2   dob_years         21453 non-null  int64  
 3   education_id      21453 non-null  int64  
 4   family_status_id  21453 non-null  int64  
 5   gender            21453 non-null  object 
 6   income_type       21453 non-null  object 
 7   debt              21453 non-null  int64  
 8   total_income      21453 non-null  int64  
 9   purpose           21453 non-null  object 
dtypes: float64(1), int64(6), object(3)
memory usage: 1.6+ MB


### Шаг 2.6. Категоризация дохода.

In [38]:
# Создание функции, которая определяет категорию в зависимости от дохода клиента
def income_group(income):
    if income < 30000:
        return 'E'
    elif 30001 <= income <= 50000:
        return 'D'
    elif 50001 <= income <= 200000:
        return 'C'
    elif 200001 <= income <= 1000000:
        return 'B'
    else:
        return 'A'
    
# Создание столбца с категориями доходов
df['total_income_category'] = df['total_income'].apply(income_group)

# Проверка после добавления столбца с категориями доходов
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,14177.753002,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.7. Категоризация целей кредита.

In [39]:
# Создание списков по категориям
#auto = ['приобретение автомобиля', 'на покупку подержанного автомобиля', 'на покупку своего автомобиля', 'автомобили', 'сделка с подержанным автомобилем', 'автомобиль', 'свой автомобиль', 'сделка с автомобилем', 'на покупку автомобиля']
#realty = ['покупка жилья', 'операции с жильем', 'покупка жилья для семьи', 'покупка недвижимости', 'покупка коммерческой недвижимости', 'покупка жилой недвижимости', 'строительство собственной недвижимости', 'недвижимость', 'строительство недвижимости', 'операции с коммерческой недвижимостью', 'строительство жилой недвижимости', 'жилье', 'операции со своей недвижимостью', 'покупка своего жилья', 'операции с недвижимостью', 'ремонт жилью', 'покупка жилья для сдачи']
#wedding = ['сыграть свадьбу', 'на проведение свадьбы', 'свадьба']
#study = ['дополнительное образование', 'образование', 'заняться образованием', 'получение образования', 'получение дополнительного образования', 'получение высшего образования', 'профильное образование', 'высшее образование', 'заняться высшим образованием']

# Создание функции, которая определяет категорию в зависимости от целей кредита
def porpose_group(porpose_name):
    if 'авто' in porpose_name:
        return 'операции с автомобилем'
    elif 'жиль' or 'недвижимост' in porpose_name:
        return 'операции с недвижимостью'
    elif 'свадьб' in porpose_name:
        return 'проведение свадьбы'
    elif 'образовани' in porpose_name:
        return 'получение образования'
    else:
        return 'категория не определена'
    
# Создание столбца с категориями по целям кредитования
df['purpose_category'] = df['purpose'].apply(porpose_group)

# Проверка после добавления столбца с категориями доходов
df.head()

# Проверка на отсутствие 'категория не определена'
df[df['purpose_category']=='категория не определена']['purpose_category'].count()

0

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [40]:
# Расчет конверсии задолжностей в зависимости от количества детей
df.pivot_table(index='children', values='debt', aggfunc=['count', 'mean'])

,count,mean
,debt,debt
children,,
0,14090,0.075444
1,4855,0.091658
2,2128,0.094925
3,330,0.081818
4,41,0.097561
5,9,0.000000


**Вывод 1:** клиенты с детьми (от 1 до 4) имеют большую конверсию задолжностей, чем бездетные. При этом, если есть 5 детей, то задолжностей нет.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [41]:
# Объединение декомпозитной таблицы со словарем семейного положения
df_family = df.merge(family_status_dict, on='family_status_id', how='left')

# Расчет конверсии задолжностей в зависимости от семейного положения
df_family.groupby('family_status')['debt'].sum() / df_family.groupby('family_status')['debt'].count()

family_status
в разводе                0.071130
вдовец / вдова           0.065693
гражданский брак         0.093494
женат / замужем          0.075452
не женат / не замужем    0.097509
Name: debt, dtype: float64

**Вывод 2:** наибольшая конверсия задолжностей наблюдается у не женатых / не замужних клиентов и находящихся в гражданском браке. Средние значения конверсии задолжностей у тех, кто официально состоит в браке или разведен. Наименьшие значения - вдовцы / вдовы.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [42]:
# Расчет конверсии задолжностей в зависимости от уровня дохода
df.pivot_table(index='total_income_category', columns='income_type', values='debt', aggfunc=['count', 'mean'], margins=True, fill_value=0)

count                                            \
income_type           безработный в декрете госслужащий компаньон пенсионер   
total_income_category                                                         
A                               0         0           0        15         0   
B                               1         0         369      1741       552   
C                               1         1        1063      3296      3097   
D                               0         0          24        25       165   
E                               0         0           1         1        15   
All                             2         1        1457      5078      3829   

                                                                      mean  \
income_type           предприниматель сотрудник студент    All безработный   
total_income_category                                                        
A                                   0        10       0     25         0.0   
B                                   1      2377       0   5041         0.0   
C                                   1      8555       1  16015         1.0   
D                                   0       136       0    350         0.0   
E                                   0         5       0     22         0.0   
All                                 2     11083       1  21453         0.5   

                                                                 \
income_type           в декрете госслужащий компаньон пенсионер   
total_income_category                                             
A                             0    0.000000  0.133333  0.000000   
B                             0    0.048780  0.063182  0.056159   
C                             1    0.063029  0.079794  0.056829   
D                             0    0.041667  0.040000  0.042424   
E                             0    0.000000  0.000000  0.133333   
All                           1    0.059025  0.074045  0.056412   

                                                                   
income_type           предприниматель сотрудник студент       All  
total_income_category                                              
A                                   0  0.000000       0  0.080000  
B                                   0  0.082878       0  0.070621  
C                                   0  0.099591       0  0.084920  
D                                   0  0.088235       0  0.060000  
E                                   0  0.000000       0  0.090909  
All                                 0  0.095732       0  0.081154

**Вывод 3:** наибольшая конверсия задолжностей наблюдается у категории "E" (с доходом до 30 000) в основном пенсионеры. Наименьшая конверсия задолжностей у категории "D" (доход от 30 001 до 50 000). Примечательно, что по типу занятости во всех категориях выделяются "сотрудники" с большими значениями конверсии задолжностей.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [43]:
# Расчет конверсии задолжностей в зависимости от цели кредита
df.groupby('purpose_category')['debt'].sum() / df.groupby('purpose_category')['debt'].count()

purpose_category
операции с автомобилем      0.093590
операции с недвижимостью    0.078031
Name: debt, dtype: float64

**Вывод 4:** кредиты выданные для операций с автомобилем и образованием имеют наибольшую конверсию задолжностей. Наименьшая конверсия задолжностей у операций с недвижимостью и проведением свадьбы.

### Общий вывод:

**Семейное положение и количество детей клиента влияет на факт погашения кредита в срок:**
* Не женатые / не замужние клиенты, а так же клиенты состоящие в гражданском браке более склонны к наличию задолжностей в отличии от вдовцов / вдов.
* Клиенты с детьми (от 1 до 4) более склонны к наличию задолжностей в отличии от бездетных. При этом, если есть 5 детей, то задолжностей нет.

Чтобы снизить количество задолжностей при выдаче кредитов необходимо отдавать приоритет бездетным вдовцам / вдовам и клиентам с 5-ю детьми.